# Procesamiento de Datos a Gran Escala

In [1]:
import findspark
findspark.init()
from pyspark import SparkConf
from pyspark.sql import SparkSession, SQLContext

In [2]:
# Configuración local
configuraDierickBrochero = (
    SparkConf()
        .set("spark.scheduler.mode", "FAIR")
        .set("spark.executor.memory", "10G")
        .set("spark.executor.cores", "1")
        .set("spark.cores.max", "9")
        .set("spark.ui.port", "4040")
        .setMaster("local[*]")  # <---- Spark local usando todos los cores
        .setAppName("hpcsparkDierickBrochero")
)

# Crear sesión Spark local
sparkDierickBrochero = (
    SparkSession
        .builder
        .config(conf=configuraDierickBrochero)
        .getOrCreate()
)

sqlContext = SQLContext(sparkDierickBrochero.sparkContext)

sparkDierickBrochero

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/19 22:35:49 WARN Utils: Your hostname, MacBook-Pro-de-Dierick.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.2 instead (on interface en0)
25/11/19 22:35:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/19 22:35:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/19 22:35:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
/Users/dierickbr/python_envs/general/lib/python3.13/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


<p><strong>Objetivo: </strong> El objetivo de este cuaderno es crear un flujo sencillo de machine learning</p>

## Cargar de datos en un Dataframe

Este conjunto de datos consta de una etiqueta categórica con dos valores (buenos o malos), una variable categórica (color) y dos variables numéricas.

Si bien los datos son sintéticos, imaginemos que este conjunto de datos representa la salud del cliente de una empresa. La columna "color" representa una calificación de salud categórica hecha por un representante de servicio al cliente. La columna "laboratorio" representa la verdadera salud del cliente. Los otros dos valores son algunas medidas numéricas de actividad dentro de una aplicación (por ejemplo, minutos de permanencia en el sitio y compras).

Supongamos que queremos entrenar un modelo de clasificación en el que esperamos predecir una variable binaria, la etiqueta, a partir de los otros valores.

In [3]:
df = sparkDierickBrochero.read.json("/databricks-datasets/definitive-guide/data/simple-ml")
df.orderBy("value2").show()

25/11/19 22:36:28 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: /databricks-datasets/definitive-guide/data/simple-ml.
java.io.FileNotFoundException: File /databricks-datasets/definitive-guide/data/simple-ml does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:917)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1238)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:907)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:56)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:381)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource.org$apache$spark$sql$catalyst$analysis$ResolveDataSource$$loadV1BatchSource(ResolveDataSource.s

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/databricks-datasets/definitive-guide/data/simple-ml. SQLSTATE: 42K03

## Transformaciones

El conjunto de datos actual no cumple con el requisito de estar en formato de Vector y, por lo tanto, debemos transformarlo al formato adecuado.

Para lograr esto en nuestro ejemplo, vamos a especificar una RFormula. Este es un lenguaje declarativo para especificar transformaciones de aprendizaje automático y es fácil de usar una vez que comprende la sintaxis.

Los operadores básicos de RFormula son:
<p>
<p>"~" Destino y términos separados</p>
<p>"+" Términos de Concat; "+ 0" significa eliminar la intersección (esto significa que la intersección y de la línea que ajustaremos será 0)</p>
<p>"-" Eliminar un término; "- 1" significa eliminar la intersección (esto significa que la intersección y de la línea que vamos a ajustar será 0; sí, esto hace lo mismo que "+ 0"</p>
<p>":" Interacción (multiplicación de valores numéricos o valores categóricos binarizados)</p>
<p>"." Todas las columnas excepto la variable objetivo / dependiente</p>
</p>

Para especificar transformaciones con esta sintaxis, necesitamos importar la clase RFormula. Luego pasamos por el proceso de definir nuestra fórmula. En este caso, queremos usar todas las variables disponibles (el ".") Y también agregar las interacciones entre valor1 y color y valor2 y color, tratándolas como características nuevas:

In [ ]:
from pyspark.ml.feature import RFormula
supervised = RFormula(formula="lab ~ . + color:value1 + color:value2")

El siguiente paso es ajustar el transformador RFormula a los datos para que descubra los posibles valores de cada columna.

No todos los transformadores tienen este requisito, pero debido a que RFormula manejará automáticamente las variables categóricas por nosotros, necesita determinar qué columnas son categóricas y cuáles no, así como cuáles son los valores distintos de las columnas categóricas.

Por esta razón, tenemos que llamar al método fit. Una vez que llamamos a fit, devuelve una versión "entrenada" de nuestro transformador que luego podemos usar para transformar nuestros datos.

Luego llamamos a transform en ese objeto para transformar nuestros datos de entrada en los datos de salida esperados.

In [ ]:
fittedRF = supervised.fit(df) # Ajusta
preparedDF = fittedRF.transform(df) # Transforma
preparedDF.show()

+-----+----+------+------------------+--------------------+-----+
color| lab|value1| value2| features|label|
+-----+----+------+------------------+--------------------+-----+
green|good| 1|14.386294994851129|(10,[1,2,3,5,8],[...| 1.0|
 blue| bad| 8|14.386294994851129|(10,[2,3,6,9],[8....| 0.0|
 blue| bad| 12|14.386294994851129|(10,[2,3,6,9],[12...| 0.0|
green|good| 15| 38.97187133755819|(10,[1,2,3,5,8],[...| 1.0|
green|good| 12|14.386294994851129|(10,[1,2,3,5,8],[...| 1.0|
green| bad| 16|14.386294994851129|(10,[1,2,3,5,8],[...| 0.0|
 red|good| 35|14.386294994851129|(10,[0,2,3,4,7],[...| 1.0|
 red| bad| 1| 38.97187133755819|(10,[0,2,3,4,7],[...| 0.0|
 red| bad| 2|14.386294994851129|(10,[0,2,3,4,7],[...| 0.0|
 red| bad| 16|14.386294994851129|(10,[0,2,3,4,7],[...| 0.0|
 red|good| 45| 38.97187133755819|(10,[0,2,3,4,7],[...| 1.0|
green|good| 1|14.386294994851129|(10,[1,2,3,5,8],[...| 1.0|
 blue| bad| 8|14.386294994851129|(10,[2,3,6,9],[8....| 0.0|
 blue| bad| 12|14.386294994851129|(10,[2,3,6,9],[12...| 0.0|
green|good| 15| 38.97187133755819|(10,[1,2,3,5,8],[...| 1.0|
green|good| 12|14.386294994851129|(10,[1,2,3,5,8],[...| 1.0|
green| bad| 16|14.386294994851129|(10,[1,2,3,5,8],[...| 0.0|
 red|good| 35|14.386294994851129|(10,[0,2,3,4,7],[...| 1.0|
 red| bad| 1| 38.97187133755819|(10,[0,2,3,4,7],[...| 0.0|
 red| bad| 2|14.386294994851129|(10,[0,2,3,4,7],[...| 0.0|
+-----+----+------+------------------+--------------------+-----+
only showing top 20 rows

En la salida podemos ver el resultado de nuestra transformación: una columna llamada características que tiene nuestros datos sin procesar.

Lo que sucede detrás de escena es bastante simple: RFormula inspecciona nuestros datos durante la llamada de ajuste y genera un objeto que transformará nuestros datos de acuerdo con la fórmula especificada.

Cuando usamos este transformador, Spark convierte automáticamente nuestra variable categórica en Dobles para que podamos ingresarla en un modelo de aprendizaje automático.

En particular, asigna un valor numérico a cada color posible.
categoría, crea características adicionales para las variables de interacción entre colores y valor1 / valor2, y las coloca todas en un solo vector.

Creemos ahora un conjunto de prueba simple basado en una división aleatoria de los datos:

In [ ]:
train, test = preparedDF.randomSplit([0.7, 0.3])

## Estimators

En este caso usaremos un algoritmo de clasificación llamado regresión logística.

Para crear nuestro clasificador, creamos una instancia de LogisticRegression, usando la configuración predeterminada o los hiperparámetros.

Luego configuramos las columnas de etiquetas y las columnas de características.

In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="label",featuresCol="features")

Antes de comenzar a entrenar este modelo, inspeccionemos los parámetros.

Este método muestra una explicación de todos los parámetros para la implementación de Spark de la regresión logística.

El método "explainParams" existe en todos los algoritmos disponibles en MLlib.

In [ ]:
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The bounds vector size must beequal with 1 for binomial regression, or the number oflasses for multinomial regression. (undefined)
maxBlockSizeInMB: maximum memory in MB for stacking input data into blocks. Data is stacked within partitions. If more than remaining data size in a partition then it is adjusted to the data size. Default 0.0 represents choosing optimal value, depends on specific algorithm. Must be >= 0. (default: 0.0)
maxIter: max number of iterations (>= 0). (default: 100)
predictionCol: prediction column name. (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities. (default: probability)
rawPredictionCol: raw prediction (a.k.a. confidence) column name. (default: rawPrediction)
regParam: regularization parameter (>= 0). (default: 0.0)
standardization: whether to standardize the training features before fitting the model. (default: True)
threshold: Threshold in binary classification prediction, in range [0, 1]. If threshold and thresholds are both set, they must match.e.g. if threshold is p, then thresholds must be equal to [1-p, p]. (default: 0.5)
thresholds: Thresholds in multi-class classification to adjust the probability of predicting each class. Array must have length equal to the number of classes, with values > 0, excepting that at most one value may be 0. The class with largest value p/t is predicted, where p is the original probability of that class and t is the class's threshold. (undefined)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 1e-06)
upperBoundsOnCoefficients: The upper bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
upperBoundsOnIntercepts: The upper bounds on intercepts if fitting under bound constrained optimization. The bound vector size must be equal with 1 for binomial regression, or the number of classes for multinomial regression. (undefined)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)

Al crear una instancia de un algoritmo no entrenado, llega el momento de ajustarlo a los datos (entrenarlo). En este caso, esto devuelve un LogisticRegressionModel.

Este código iniciará un trabajo de Spark para entrenar el modelo. A diferencia de las transformaciones, el ajuste de un modelo de aprendizaje automático es ansioso y se realiza de inmediato.

In [ ]:
fittedLR = lr.fit(train)

Una vez completado, puede usar el modelo para hacer predicciones. Lógicamente, esto significa transformar características en etiquetas.

Hacemos predicciones con el método transform. Por ejemplo, podemos transformar nuestro conjunto de datos de entrenamiento para ver qué etiquetas asignó nuestro modelo a los datos de entrenamiento y cómo se comparan con los resultados reales.

Realicemos esa predicción con el siguiente fragmento de código:

In [ ]:
fittedLR.transform(train).select("label", "prediction").show(50)

+-----+----------+
label|prediction|
+-----+----------+
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
+-----+----------+
only showing top 50 rows

Nuestro siguiente paso sería evaluar manualmente este modelo y calcular métricas de rendimiento como la tasa de verdaderos positivos, la tasa de falsos negativos, etc.

## Pipelines

Como probablemente haya notado, si está realizando muchas transformaciones, escribir todos los pasos y realizar un seguimiento de DataFrames termina siendo bastante tedioso.

Por eso Spark incluye el concepto Pipeline.

Tenga en cuenta que es esencial que las instancias de transformadores o modelos no se reutilicen en diferentes Pipeline. Cree siempre una nueva instancia de un modelo antes de crear otra Pipeline.

Para asegurarnos de no sobreajustarnos, crearemos un conjunto de pruebas de holdout(un método de validación) y ajustaremos nuestros hiperparámetros en función de un conjunto de validación (tenga en cuenta que creamos este conjunto de validación basado en el conjunto de datos original, no en el preparedDF):

In [ ]:
train, test = df.randomSplit([0.7, 0.3])

Ahora que tiene un conjunto de entrenamiento y prueba, creemos las stages base en nuestra Pipeline.

Una stage simplemente representa un transformador o un estimador. En nuestro caso, tendremos dos estimadores. La RFomula y el LogisticRegresión:

In [ ]:
rForm = RFormula()
lr = LogisticRegression().setLabelCol("label").setFeaturesCol("features")

Ahora, en lugar de usar manualmente nuestras transformaciones y luego ajustar nuestro modelo, simplemente las hacemos stages en la Pipeline general, como en el siguiente fragmento de código:

In [ ]:
from pyspark.ml import Pipeline
stages = [rForm, lr]
pipeline = Pipeline().setStages(stages)

## Entrenamiento y Evaluación

Ahora que organizó la Pipeline, el siguiente paso es el Entrenamiento.

En este caso, no entrenaremos solo un modelo. Entrenaremos varias variaciones del modelo especificando diferentes combinaciones de hiperparámetros que nos gustaría que Spark probara.

Luego, seleccionaremos el mejor modelo usando un evaluador que compara sus predicciones con nuestros datos de validación.

Podemos probar diferentes hiperparámetros en toda la Pipeline, incluso en la fórmula de RF que usamos para manipular los datos sin procesar.

En nuestro ParamGridBuilder, hay tres hiperparámetros que varían de los valores predeterminados:
<li>Dos versiones diferentes de RFormula</li>
<li>Tres opciones diferentes para el parámetro ElasticNet</li>
<li>Dos opciones diferentes para el parámetro de regularización</li>
Esto nos da un total de 12 combinaciones diferentes de estos parámetros, lo que significa que entrenaremos 12 versiones diferentes de regresión logística.

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder
params = ParamGridBuilder()\
.addGrid(rForm.formula, [
"lab ~ . + color:value1",
"lab ~ . + color:value1 + color:value2"])\
.addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
.addGrid(lr.regParam, [0.1, 2.0])\
.build()

Ahora que la cuadrícula está construida, es hora de especificar nuestro proceso de evaluación. El evaluador nos permite comparar de forma automática y objetiva varios modelos con la misma métrica de evaluación.

En este caso usaremos el BinaryClassificationEvaluator, que tiene una serie de métricas de evaluación potenciales.

En este caso usaremos areaUnderROC, que es el área total bajo la característica operativa del receptor, una medida común de desempeño de clasificación.

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()\
.setMetricName("areaUnderROC")\
.setRawPredictionCol("prediction")\
.setLabelCol("label")

Ahora que tenemos una canalización que especifica cómo se deben transformar nuestros datos, realizaremos la selección del modelo para probar diferentes hiperparámetros en nuestro modelo de regresión logística y medir el éxito comparando su desempeño usando la métrica areaUnderROC.

In [ ]:
from pyspark.ml.tuning import TrainValidationSplit
tvs = TrainValidationSplit()\
.setTrainRatio(0.75)\
.setEstimatorParamMaps(params)\
.setEstimator(pipeline)\
.setEvaluator(evaluator)

Ejecutemos toda la Pipeline que construimos. Para revisar, la ejecución de esta canalización probará todas las versiones del modelo con el conjunto de validación.

In [ ]:
tvsFitted = tvs.fit(train)

/databricks/spark/python/pyspark/ml/util.py:838: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

Tambien se evalua cómo funciona el algoritmo con el conjunto de prueba:

In [ ]:
evaluator.evaluate(tvsFitted.transform(test))

0.90625